In [1]:
import pandas as pd

# Preprocessing
ratings_df = pd.read_csv('./data/ratings.dat', sep='::', header=None, engine='python')

ratings_df.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [5]:
user_rating_df = ratings_df.pivot(index='UserID', columns='MovieID', values='Rating')
norm_user_rating_df = user_rating_df.fillna(0)/5.0

norm_user_rating_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,3913,3914,3915,3916,3917,3918,3919,3920,3921,3922,3923,3924,3925,3926,3927,3928,3929,3930,3931,3932,3933,3934,3935,3936,3937,3938,3939,3940,3941,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
UserID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.8,0.0,0.8,0.0,0.6,0.0,0.0,0.6,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
import tensorflow as tf

train_x, test_x = tf.split(norm_user_rating_df.to_numpy().astype('float32'), 2)

train_dataset = tf.data.Dataset.from_tensor_slices(train_x)
train_dataset = train_dataset.shuffle(buffer_size=3020).batch(20)

test_dataset = tf.data.Dataset.from_tensor_slices(test_x)
test_dataset = test_dataset.shuffle(buffer_size=3020).batch(20)

In [0]:
from rbm_model import RBM

epochs = 12
alpha = 0.01

model = RBM(len(user_rating_df.columns), 128)

def run(dataset, train=True):
    for step, batch in enumerate(dataset):
        loss = model.train(batch, alpha) if train else model.evaluate(batch)
        
        if step % 20 == 0:
            print('step %s: loss = %s' % (step, loss))

In [20]:
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))
    run(train_dataset)

Start of epoch 0
step 0: loss = tf.Tensor(0.48841065, shape=(), dtype=float32)
step 20: loss = tf.Tensor(0.12576094, shape=(), dtype=float32)
step 40: loss = tf.Tensor(0.23763466, shape=(), dtype=float32)
step 60: loss = tf.Tensor(0.20356342, shape=(), dtype=float32)
step 80: loss = tf.Tensor(0.1908678, shape=(), dtype=float32)
step 100: loss = tf.Tensor(0.13451593, shape=(), dtype=float32)
step 120: loss = tf.Tensor(0.1967901, shape=(), dtype=float32)
step 140: loss = tf.Tensor(0.12215166, shape=(), dtype=float32)
Start of epoch 1
step 0: loss = tf.Tensor(0.13293847, shape=(), dtype=float32)
step 20: loss = tf.Tensor(0.05980464, shape=(), dtype=float32)
step 40: loss = tf.Tensor(0.11925797, shape=(), dtype=float32)
step 60: loss = tf.Tensor(0.13031788, shape=(), dtype=float32)
step 80: loss = tf.Tensor(0.1087631, shape=(), dtype=float32)
step 100: loss = tf.Tensor(0.096595794, shape=(), dtype=float32)
step 120: loss = tf.Tensor(0.09954508, shape=(), dtype=float32)
step 140: loss = tf.

In [10]:
# Evaluation
run(test_dataset, train=False)

step 0: loss = tf.Tensor(0.065865636, shape=(), dtype=float32)
step 10: loss = tf.Tensor(0.058459807, shape=(), dtype=float32)
step 20: loss = tf.Tensor(0.047166757, shape=(), dtype=float32)
step 30: loss = tf.Tensor(0.04707501, shape=(), dtype=float32)
step 40: loss = tf.Tensor(0.051068544, shape=(), dtype=float32)
step 50: loss = tf.Tensor(0.06907609, shape=(), dtype=float32)
step 60: loss = tf.Tensor(0.054579068, shape=(), dtype=float32)
step 70: loss = tf.Tensor(0.043084733, shape=(), dtype=float32)
step 80: loss = tf.Tensor(0.05219104, shape=(), dtype=float32)
step 90: loss = tf.Tensor(0.059803564, shape=(), dtype=float32)
step 100: loss = tf.Tensor(0.07351809, shape=(), dtype=float32)
step 110: loss = tf.Tensor(0.05142202, shape=(), dtype=float32)
step 120: loss = tf.Tensor(0.06538803, shape=(), dtype=float32)
step 130: loss = tf.Tensor(0.04166271, shape=(), dtype=float32)
step 140: loss = tf.Tensor(0.060998928, shape=(), dtype=float32)
step 150: loss = tf.Tensor(0.042090666, sha

In [11]:
movies_df = pd.read_csv('./data/movies.dat', sep='::', header=None, engine='python')
movies_df.columns = ['MovieID', 'Title', 'Genres']

movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [15]:
# Test the model with a mock case
mock_user_id = 176

inputUser = tf.reshape(train_x[mock_user_id - 1], (1, -1))
recommended = model.predict(inputUser)

# List of movies recommended to user 176
scored_movies_df_mock = movies_df[movies_df['MovieID'].isin(user_rating_df.columns)]
scored_movies_df_mock = scored_movies_df_mock.assign(RecommendationScore = recommended[0])

scored_movies_df_mock.head()

,MovieID,Title,Genres,RecommendationScore
0,1,Toy Story (1995),Animation|Children's|Comedy,0.0
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0.0
2,3,Grumpier Old Men (1995),Comedy|Romance,0.0
3,4,Waiting to Exhale (1995),Comedy|Drama,0.0
4,5,Father of the Bride Part II (1995),Comedy,0.0


In [16]:
scored_movies_df_mock.sort_values(["RecommendationScore"], ascending=False).head()

,MovieID,Title,Genres,RecommendationScore
3397,3466,Heart and Souls (1993),Comedy|Fantasy,1.0
3141,3210,Fast Times at Ridgemont High (1982),Comedy,1.0
220,223,Clerks (1994),Comedy,1.0
2728,2797,Big (1988),Comedy|Fantasy,1.0
2727,2796,Funny Farm (1988),Comedy,1.0
